### Real, Complex and Symplectic Reflection Groups - March 2023, RUB

## Computational Aspects of Complex Reflection Groups

Götz Pfeiffer - University of Galway

# 4. Vectors: Enumerating Modules and Hecke Algebras

![Benches](images/benches.jpg)

## Setup

First, reload the algorithms from the last day ...

In [ ]:
LoadPackage("jupyterviz");
opts := rec(vertexwidth := 12, vertexheight := 12, edgecolor := "#def");;
Read("orbits.g");
Read("coxeter.g");
Read("variants.g");
Read("enumerate.g");
Read("examples.g");

##  Spinning: $K$-Linear Orbit Algorithm

* In the **linear** version, the points of an orbit are linearly independent vectors in $K^n$.
* And the acting operators are like matrices in $K^{n\times n}$.

In [ ]:
spinning :=  function(aaa, x, under)
    local list,  a,  y,  z;
    list := [x];
    for y in list do
        for a in aaa do
            z := under(y, a);
            if not z in VectorSpace(Rationals, list) then
                Add(list, z);
            fi;
        od;
    od;
    return list;
end;

### Example

In [ ]:
A := [
[0,1,0,0,0],
[0,0,1,0,0],
[0,0,0,1,0],
[0,0,0,0,1],
-[1,1,1,1,1],
];

In [ ]:
spinning([A], [0,0,1,0,0], OnRight);

In [ ]:
spinning([A], [1,1,0,1,1], OnRight);

In [ ]:
spinning([A], [1,0,1,0,1], OnRight);

##  Spinning with Images

* We will need to be able to convert between sparse and dense representations of vectors.

In [ ]:
sparseVec:= function(vec)
    local   poss;
    poss:= PositionsProperty(vec, v -> v <> Zero(v));
    return rec(pos := poss,  val:= vec{poss});
end;

denseVec:= function(l, v)
    local   vec;
    vec := 0*[1..l];  vec{v.pos} := v.val;
    return vec;
end;

In [ ]:
vec := [0,0,1,1,0];
v := sparseVec(vec);

In [ ]:
denseVec(Length(vec), v) = vec;

* spinning with images (cf. orbit with images)

* In the linear version the images are vectors: linear combination of the base points.

In [ ]:
spinning_with_images :=  function(aaa, x, under)
    local   list,  images,  i,  y,  k,  a,  z,  v;
    list := [x];  images := List(aaa, x-> []);  i := 0;
    while i < Length(list) do
        i := i+1;  y := list[i];
        for k in [1..Length(aaa)] do
            a := aaa[k];  z := under(y, a);
            v := SolutionMat(list, z);
            if v = fail then
                Add(list, z);
                v := rec(pos := [Length(list)], val := [1]);
            else
                v := sparseVec(v);
            fi;
            images[k][i]:= v;
        od;
    od;
    return rec(list := list, images := images);
end;

### Example: Specht-Module

* Everybody knows that Specht modules are made of Standard Young Tableaus ...
* Here is a random one, of shape $\lambda = (2,2,1)$ (for the rest see the exercise on SYTs in part 1).

In [ ]:
tab := [
  [1,4],
  [2,5],
  [3],
];

* and the corresponding symmetric group

In [ ]:
N := Sum(tab, Length);;  gens := transpositions(N);

* each tableau (standard or not) yields a (row) word, recording for each number the row it is in.

In [ ]:
wordTab := function(tab)
    local   word,  i;
    word := [];
    for i in [1..Length(tab)] do
        word{tab[i]} := 0*tab[i] + i;
    od;
    return word;
end;

In [ ]:
wordTab(tab);

* the column stabilizer is the stabilizer of the column word.
* the orbit of a tableau under the column stabilizer is a (column) tabloid.

In [ ]:
colWord := wordTab(TransposedMat(tab));
stab := takeAway(orbit_with_stabilizer(gens, colWord, Permuted).stab, ());
tabs := orbit_with_transversal(stab, tab, OnTuplesTuples);

* the symmetric group acts on the row words

In [ ]:
orb := orbit_with_images(gens, wordTab(tab), Permuted);
words := orb.list;;
perms := List(orb.images, PermList);

* the polytabloid is a signed combination of tabloids, written as a vector in the permutation module.

In [ ]:
vec := List(words, x-> 0);
for i in [1..Length(tabs.list)] do
    vec[Position(words, wordTab(tabs.list[i]))] := SignPerm(tabs.reps[i]);
od;

* apply spinning to our polytabloid

In [ ]:
vvv:= spinning_with_images(perms, vec, Permuted);

* How to convert a list of sparse vectors into a matrix (cf. `PermList`)

In [ ]:
mat_list:= list -> List(list, v -> denseVec(Length(list), v));

In [ ]:
mats := List(vvv.images, mat_list);

In [ ]:
for m in mats do PrintArray(m);  Print("\n"); od;

## Schreier Matrices and the Group Algebra

* Another way to produce matrices from an orbit is to read the resulting permutations as **permutation matrices**.

In [ ]:
gens := transpositions(4);;
orb := orbit_with_images(gens, 4, OnPoints);;
mats := List(orb.images, x -> PermutationMat(PermList(x), Length(x)));

In [ ]:
for m in mats do PrintArray(m);  Print("\n"); od;

* Next, we use the Schreier generators as entries in those matrices, rather than just $1$.
* Once again, only a small modification of an earlier version of the orbit algorithm, `orbit_with_images`, is needed.
* Storing these images as **sparse vectors** will allow to use `mat_list` to recover the matrices.

In [ ]:
orbit_with_schreier:= function(aaa, x, under)
    local   list,  reps,  i,  images,  y,  k,  a,  z,  l;
    list := [x];  reps := [()];  i := 0;
    images := List(aaa, x -> []);
    while i < Length(list) do
        i := i+1;  y := list[i];
        for k in [1..Length(aaa)] do
            a := aaa[k];  z := under(y, a);
            l := Position(list, z);
            if l = fail then
                Add(list, z);
                Add(reps, reps[i] * a);
                l := Length(list);
            fi;
            images[k][i] := rec(pos := [l], val := [reps[i] * a / reps[l]]);
        od;
    od;
    return rec(list := list, images := images);
end;

In [ ]:
gens := transpositions(4);;
orb := orbit_with_schreier(gens, 4, OnPoints);
mats := List(orb.images, mat_list);

In [ ]:
for m in mats do PrintArray(m);  Print("\n"); od;

* These matrices form a representation of the group algebra $\mathbb{C}G$, acting regularly on $\mathbb{C}G$ regarded as a right $\mathbb{C}G$-module over the group algebra $\mathbb{C}H$ of the stabilizer $H$:
$$
  \mathbb{C}G = \bigoplus_{t \in T} \mathbb{C}H t,
$$
where $T$ is the transversal.

* Let's call the resulting matrices the **Schreier matrices** of the action.
* If we tell GAP what this means for sums and products of permutations and zero, the Schreier matrices can even be multiplied.

In [ ]:
InstallMethod(\*, "for 0 and perm", true, [IsInt, IsPerm], 0, function(zero, perm)
    return zero;
end);
InstallMethod(\*, "for perm and 0", true, [IsPerm, IsInt], 0, function(perm, zero)
    return zero;
end);
InstallTrueMethod( IsAdditiveElementWithInverse, IsPerm );
InstallMethod(\+, "for 0 and perm", true, [IsInt, IsPerm], 0, function(zero, perm)
    return perm;
end);
InstallMethod(\+, "for perm and 0", true, [IsPerm, IsInt], 0, function(perm, zero)
    return perm;
end);

In [ ]:
mats[1] * mats[1];
mats[1] * mats[2];
mats[1] * mats[2] * mats[1] = mats[2] * mats[1] * mats[2];

* The Schreier matrices can serve as a blueprint for induced representations: Take any matrix representation of $H$ and replace the Schreier generators $h \in H$ in the Schreier matrix of $g \in G$ by the matrices representing $h$:  the result (with a suitable interpretation of $0$) will be a matrix representing $g$. Replacing all Schreier generators by the trivial representation $1$ yields the above  permutation representation of $G$ on the cosets of $H$.

* In the case of a Coxeter group acting on the cosets of a parabolic subgroup, the Schreier generators are either trivial, or simple reflections, thanks to the following result.

<div class="alert alert-danger">

**Theorem** (Deodhar's Lemma) **.**  Let $(W, S)$ be a finite Coxeter group, and let $J \subseteq S$.
    
* Let $x \in X_J$ and $s \in S$.  Then either $xs \in X_J$ or $xs = ux$ for some $u \in J$.
    
</div>

* Note how $u = xsx^{-1}$ is a Schreier generator if $xs \notin X_J$.

##  Iwahori-Hecke Algebra

* The Iwahori-Hecke algebra $H$ of $(W, S)$ is ...

<div class="alert alert-danger">

**Theorem** (Deodhar's Lemma for Iwahori-Hecke algebras) **.**  Let $(W, S)$ be a finite Coxeter group, and the $J \subseteq S$.
    
* Let $x \in X_J$ and $s \in S$.  Then either 
$$
T_x T_s = \begin{cases}
(q-1)T_x + qT_{xs}, & \text{if } \ell(xs) < \ell(x),\\
T_{xs}, & \text{if } \ell(xs) > \ell(x) \text { and } xs \in X_J,\\
T_u T_x, & \text{if } \ell(xs) > \ell(x) \text { and } xs \notin X_J,
\end{cases}
$$
for some $u \in J$. 
</div>

* We thus get Schreier matrices for the generators $T_s$ of $H$.  For example: ...
$$
T_1 \mapsto  \left[ \begin{array}{cccc}
      T_{1}&.&.&.\\ .&T_{1}&.&.\\ .&.&.&1\\ .&.&q&q{-}1
    \end{array} \right],
    \quad
T_2 \mapsto  \left[ \begin{array}{cccc}
      T_{2}&.&.&.\\ .&.&1&.\\ .&q&q{-}1&\\ .&.&.&T_{1}
    \end{array} \right],
    \quad
T_3 \mapsto  \left[ \begin{array}{cccc}
      .&1&.&.\\ q&q{-}1&.&.\\ .&.&T_{2}&.\\ .&.&.&T_{2}
    \end{array} \right]
$$

* As coset table, aka image list of sparse vectors.
$$
  \begin{array}{l|ccc}
    x &x.1&x.2&x.3\\\hline
    x_1 = \emptyset & 1 \cdot x & 2 \cdot x & \underline{x_2}\\
    x_2 = 3 & 1 \cdot x & \underline{x_3} & (q{-}1)x + qx_1 \\
    x_3 = 32 & \underline{x_4} & (q{-}1) x + qx_2 & 2 \cdot x \\
    x_4 = 321 & (q{-}1)x + q x_3 & 1 \cdot x & 2 \cdot x
  \end{array}
$$

* Can we find such a coset table for the Hecke algebra of a complex reflection group? Yes ...

##  Linear Coset Enumeration

* Like the spinning algorithm is a linear version of the orbit algorithm, there is a lineasr version of the coset enumeration procedure.  Naturally, at certain stages of the procedure, a linear result has to be expected, and handled.

* ... details omitted ...

* Example $G(3,3,3)$.

In [ ]:
G:= G333;

In [ ]:
data := enumerate(G);

In [ ]:
nodes := Filtered(data.list, isActive);

In [ ]:
gens := List(G.gens, i -> Sortex(List(nodes, x -> flat(x!.next[i])!.idx)));

In [ ]:
sizeOfGroup(GroupWithGenerators(gens));

In [ ]:
edges := Union(List(nodes, node -> List(node!.next, x -> [node!.idx, flat(x)!.idx])));;
PlotGraph(Filtered(edges, x -> x[1] <> x[2]), opts);

$$
  \begin{array}{l|cccc}
    x &x.t_0&x.t_1&x.t_2&x.s_3\\\hline
    x_0 =  & t_0 \cdot x & t_1 \cdot x & t_2 \cdot x & \underline{x_1} \\
    x_1 = s_3 & \underline{x_2} & \underline{x_3} & \underline{x_4} & (q{-}1) x + q x_0\\
    x_2 = s_3 t_0 & (q{-}1) x + q x_1 & \eqref{2.1} & x_5 & t_0 \cdot x_2 \\
    x_3 = s_3 t_1 & \underline{x_5} & (q{-}1) x + q x_1 & \eqref{3.2} & t_1 \cdot x_3 \\
    x_4 = s_3 t_2 & \underline{x_6} & x_5 & (q{-}1) x + q x_1 & t_2 \cdot x_4 \\
    x_5 = s_3 t_1 t_0 & (q{-}1) x + q x_3 & (q{-}1) x + q x_4 & (q{-}1) x + q x_2 & \underline{x_7} \\
    x_6 = s_3 t_2 t_0 & (q{-}1) x + q x_4 & \eqref{6.1} & (q{-}1) x_5 + q x_3 & \underline{x_8} \\
    x_7 = s_3 t_1 t_0 s_3 & t_1 \cdot x & t_2 \cdot x & t_0 \cdot x & (q{-}1) x + q x_5 \\
    x_8 = s_3 t_2 t_0 s_3 & t_2 \cdot x & \eqref{8.1} & \eqref{8.2} & (q{-}1) x + q x_6\\
  \end{array}
$$

* where
  * $x_2.t_1 = (q{-}1) x_3 + (1{-}q) x_4 + x_6$
  * $x_3.t_2 = (1{-}q) x_2 + (q{-}1) x_3 + x_6$
  * $  x_6.t_1 
  = (q{-}1) x_5 + (1{-}q)(q{-}1) x_4 + (q{-}1) x
  + (1{-}q) q x_1 + q x_2$
  * $  x_8.t_1 =
  (q{-}1)t_2\cdot x_5
  + q(q{-}1)t_2t_0' \cdot x_3
  +  t_0 \cdot x_8
  + (1{-}q)t_0t_2 \cdot x_4
  + q(1{-}q)t_1 \cdot x_1$
  * $  x_8.t_2 =
   q(q{-}1) x_3
   + (q{-}1)t_0 \cdot x_5
   + t_1 \cdot x_8
   + (1{-}q)t_1 \cdot x_6
   + q(1{-}q)t_1t_0' \cdot x_2$

## Exercises, etc.

* Expand `spinning` into `spinning_with_matrices` ...

* ...